In [ ]:
using QuantumCollocation
using NamedTrajectories
using TrajectoryIndexingUtils

using Interpolations
using LinearAlgebra
using SparseArrays

# Plots
using CairoMakie
using Colors

In [ ]:
const Units = 1e9
const MHz = 1e6 / Units
const GHz = 1e9 / Units
const ns = 1e-9 * Units
const μs = 1e-6 * Units
;

In [ ]:
# Operators
const Paulis = Dict(
    "I" => Matrix{ComplexF64}(I, 2, 2),
    "X" => Matrix{ComplexF64}([0 1; 1 0]),
    "Y" => Matrix{ComplexF64}([0 -im; im 0]),
    "Z" => Matrix{ComplexF64}([1 0; 0 -1]),
)
const a = [0 1; 0 0]
const ad = transpose(a);

In [ ]:
excitation(theta) = exp(-im/2 * theta * (Paulis["Z"]))

In [ ]:
H_drift = [ ]
H_drives = [
     a + ad,
     im * (a - ad),
]
system = QuantumSystem(H_drives);

# Times
t_f = 10 * ns
n_steps = 51
times = range(0, t_f, n_steps)
Δt = times[2] - times[1]

# Inspect previous results

In [ ]:
trajs = []
for i in 1:10
    push!(trajs, load_traj("playground/AndySamples/data_point_$i.jld2"))
end

θs = range(0,pi/5,10)
excitation(θs[end])

In [ ]:
Δas = []
for (a, b) in zip(trajs[1:end-1], trajs[2:end])
    push!(Δas, b[:a] - a[:a])
end
Δas[1]

f = Figure()
ax = Axis(f[1, 1])
ax1 = Axis(f[2, 1])
# ten-color gradient from light to dark
colors = reverse([HSV(0.0, 0.0, i) for i in range(0.2, 0.8, length(Δas))])
for (i, Δa) in enumerate(Δas)
    lines!(ax, Δa[1, :], color=colors[i], linewidth=5)
    lines!(ax1, Δa[2, :], color=colors[i], linewidth=5)
end
f

In [ ]:
ΔUs = []
for (a, b) in zip(trajs[1:end-1], trajs[2:end])
    push!(ΔUs, b[:Ũ⃗] - a[:Ũ⃗])
end
Δas[1]

f = Figure()
axes = [Axis(f[1, 1]), Axis(f[1, 2]), Axis(f[2, 1]), Axis(f[2, 2])]
# ten-color gradient from light to dark
colors = reverse([HSV(0.0, 0.0, i) for i in range(0.2, 0.8, length(ΔUs))])
for (i, ΔU) in enumerate(ΔUs)
    for j in 1:4
        lines!(axes[j], ΔU[j, :], color=colors[i], linewidth=5)
    end
end
f

In [ ]:
plot(trajs[8])

In [ ]:
plot(trajs[10])

In [ ]:
plot(trajs[1])

# Generate new results

In [6]:
# Shape the cost function with weights on states and controls
Q = 100.
R = 1e-4

# Add control bounds
a_bound = 2 * π * 100 * MHz
dda_bound = .05
initial_θs = range(0, pi/2, 5)
initial_infidelities = []
initial_trajectories = []
initial_probs = []

ops = Options()
ops.recalc_y = "yes"
ops.recalc_y_feas_tol = 1e-1
ops.print_level = 2

for θ ∈ initial_θs
    print("\nAngle $θ\n")
    target = excitation(θ)

    U_guess = nothing
    if(length(initial_trajectories) > 0)
        U_guess = initial_trajectories[length(initial_trajectories)][:Ũ⃗]
    end

    p = UnitarySmoothPulseProblem(
        system,
        target,
        n_steps,
        Δt;
        a_bound=a_bound,
        dda_bound=dda_bound,
        Q=Q,
        R=R,
        verbose=false,
        hessian_approximation=true,
        pade_order=10,
        free_time=false,
        timesteps_all_equal=true,
        ipopt_options=ops,
    )
    println("Solving...")
    solve!(p; max_iter=100)
    infid_θ = 1 - unitary_fidelity(p)
    println("\tInfidelity: $infid_θ\n")
    push!(initial_infidelities, infid_θ)
    push!(initial_trajectories, copy(p.trajectory))
    push!(initial_probs, p)
end



Run a single free time problem, for later.

In [ ]:
θ = π/5
print("\nAngle $θ\n")
target = excitation(θ)

p_free = UnitarySmoothPulseProblem(
    system,
    target,
    n_steps,
    Δt;
    a_bound=a_bound,
    dda_bound=dda_bound,
    Q=Q,
    R=R,
    verbose=false,
    hessian_approximation=true,
    pade_order=10,
    free_time=true,
    timesteps_all_equal=false,
    ipopt_options=ops,
)

println("Solving...")
solve!(p_free; max_iter=100)
infid_θ = 1 - unitary_fidelity(p_free)
println("\tInfidelity: $infid_θ\n")

We can pursue coordinated re-optimization by moving a point closer to its neighbors.

In one dimension, three points might be necessary to enforce a majority vote type of structure.

In higher dimensions, I am not sure.

# New joint optimization objective

We will start with optimizing a pair of states.

In [ ]:
function ⊕(A::AbstractMatrix, B::AbstractMatrix)
    return [A zeros((size(A, 1), size(B, 2))); zeros((size(B, 1), size(A, 2))) B]
end

function ⊕(Ã⃗::AbstractVector, B̃⃗::AbstractVector)
    return operator_to_iso_vec(iso_vec_to_operator(Ã⃗) ⊕ iso_vec_to_operator(B̃⃗))
end

function ⊕(sys₁::QuantumSystem, sys₂::QuantumSystem; R::DataType=Float64)
    H_drift_real = sys₁.H_drift_real ⊕ sys₂.H_drift_real
    H_drift_imag = sys₁.H_drift_imag ⊕ sys₂.H_drift_imag
    HZ₁ = spzeros(size(sys₁.H_drift_real))
    HZ₂ = spzeros(size(sys₂.H_drift_real))
    H_drives_real = [
        [H ⊕ HZ₂ for H ∈ sys₁.H_drives_real]...,
        [HZ₁ ⊕ H for H ∈ sys₂.H_drives_real]...
    ]
    H_drives_imag = [
        [H ⊕ HZ₂ for H ∈ sys₁.H_drives_imag]...,
        [HZ₁ ⊕ H for H ∈ sys₂.H_drives_imag]...
    ]
    return QuantumSystem{R}(
        H_drift_real,
        H_drift_imag,
        H_drives_real,
        H_drives_imag,
        QuantumSystems.G(H_drift_real + im * H_drift_imag),
        QuantumSystems.G.(H_drives_real .+ im * H_drives_imag),
        merge(sys₁.params, sys₂.params)
    )
end


function ⊕(traj₁::NamedTrajectory, traj₂::NamedTrajectory)
    # TODO: Free time problem
    if traj₁.timestep isa Symbol || traj₂.timestep isa Symbol
        throw(ErrorException("Free time problems not supported"))
    end

    if traj₁.timestep != traj₂.timestep
        throw(ErrorException("Timesteps must be equal"))
    end

    components = (
        Ũ⃗ = stack(
            map(zip(eachcol(traj₁[:Ũ⃗]), eachcol(traj₂[:Ũ⃗]))) do (c1, c2)
                c1 ⊕ c2
            end
        ),
        a = vcat(traj₁[:a], traj₂[:a]),
        da = vcat(traj₁[:da], traj₂[:da]),
        dda = vcat(traj₁[:dda], traj₂[:dda]),
    )

    bounds = (
        a = vcat.(traj₁.bounds.a, traj₂.bounds.a),
        dda = vcat.(traj₁.bounds.a, traj₂.bounds.a),
    )

    initial = (
        Ũ⃗ = traj₁.initial[:Ũ⃗] ⊕ traj₂.initial[:Ũ⃗],
        a = vcat(traj₁.initial[:a], traj₂.initial[:a])
    )

    final = (
        a = vcat(traj₁.final[:a], traj₂.final[:a]),
    )

    goal = (
        Ũ⃗ = traj₁.goal[:Ũ⃗] ⊕ traj₂.goal[:Ũ⃗],
    )

    return NamedTrajectory(
        components;
        controls=(:dda,),
        timestep=traj₁.timestep,
        bounds=bounds,
        initial=initial,
        final=final,
        goal=goal
    )
end


TODO: write pairwise cost
1. You want to use $\vec{Z}$ for the Δt (not the shaper Z)
2. You want to allow for timesteps that are arbitrary, not commit to all timesteps.
3. There won't be timestep slices if Δt is not part of the control

In [ ]:
function UnitaryPairwiseQuadraticRegularizer(
    Q::AbstractVector{<:Real},
    times::AbstractVector{Int},
    Ũ⃗₁_indices::AbstractVector{Int},
    Ũ⃗₂_indices::AbstractVector{Int};
    name::Symbol=:Ũ⃗,
    timestep_symbol::Symbol=:Δt,
    eval_hessian::Bool=false,
)
    params = Dict(
        :type => :UnitaryPairwiseQuadraticRegularizer,
        :times => times,
        :name => name,
        :Q => Q,
        :eval_hessian => eval_hessian,
    )

    @views function L(Z⃗::AbstractVector{<:Real}, Z::NamedTrajectory)
        J = 0.0
        for t ∈ times
            if Z.timestep isa Symbol
                Δt = Z⃗[slice(t, Z.components[timestep_symbol], Z.dim)]
            else
                Δt = Z.timestep
            end
            
            Ũ⃗ₜ = Z⃗[slice(t, Z.components[name], Z.dim)]
            rₜ = Δt .* (Ũ⃗ₜ[Ũ⃗₁_indices] .- Ũ⃗ₜ[Ũ⃗₂_indices])
            J += 0.5 * rₜ' * (Q .* rₜ)
        end
        return J
    end

    @views function ∇L(Z⃗::AbstractVector{<:Real}, Z::NamedTrajectory)
        ∇ = zeros(Z.dim * Z.T)        
        Threads.@threads for t ∈ times
            Ũ⃗ₜ_slice = slice(t, Z.components[name], Z.dim)
            Ũ⃗₁_slice_indices = Ũ⃗ₜ_slice[Ũ⃗₁_indices]
            Ũ⃗₂_slice_indices = Ũ⃗ₜ_slice[Ũ⃗₂_indices]
            Ũ⃗₁ = Z⃗[Ũ⃗₁_slice_indices]
            Ũ⃗₂ = Z⃗[Ũ⃗₂_slice_indices]

            if Z.timestep isa Symbol
                Δt_slice = slice(t, Z.components[timestep_symbol], Z.dim)
                Δt = Z⃗[Δt_slice]
                ∇[Δt_slice] .= (Ũ⃗₁ .- Ũ⃗₂)' * (Q .* (Δt .* (Ũ⃗₁ .- Ũ⃗₂)))
            else
                Δt = Z.timestep
            end

            ∇[Ũ⃗₁_slice_indices] .= Q .* (Δt.^2 .* (Ũ⃗₁ .- Ũ⃗₂))
            ∇[Ũ⃗₂_slice_indices] .= Q .* (Δt.^2 .* (Ũ⃗₂ .- Ũ⃗₁))
        end
        return ∇
    end

    ∂²L = nothing
    ∂²L_structure = nothing

    if eval_hessian
        throw(ErrorException("Hessian not implemented"))
    end

    return Objective(L, ∇L, ∂²L, ∂²L_structure, Dict[params])
end


function UnitaryPairwiseQuadraticRegularizer(
    traj::NamedTrajectory,
    Q::AbstractVector{<:Real};
    Ũ⃗₁_indices::Union{AbstractVector{Int}, Nothing}=nothing,
    Ũ⃗₂_indices::Union{AbstractVector{Int}, Nothing}=nothing,
    name::Symbol=:Ũ⃗,
    kwargs...
)
    # Each Ũ⃗ in the pair has size Q (≈ handles floats returned by reshapes)
    # Default assumption is there is only one pair in Ũ⃗
    if isnothing(Ũ⃗₁_indices)
        Ũ⃗₁_indices = findall(ones(length(Q)) ⊕ zeros(length(Q)) .≈ 1)
    end
    if isnothing(Ũ⃗₂_indices)
        Ũ⃗₂_indices = findall(zeros(length(Q)) ⊕ ones(length(Q)) .≈ 1)
    end

    return UnitaryPairwiseQuadraticRegularizer(
        Q, 
        1:traj.T,
        Ũ⃗₁_indices,
        Ũ⃗₂_indices,
        name=name, 
        kwargs...
    )
end

function UnitaryPairwiseQuadraticRegularizer(
    traj::NamedTrajectory,
    Q::Real;
    name::Symbol=:Ũ⃗,
    kwargs...
)   
    # Undo real/imag, undo vectorization
    # Select block
    # Redo vectorization, redo real/imag
    dim = (isqrt(traj.dims[name] ÷ 2) ÷ 2)^2 * 2
    return UnitaryPairwiseQuadraticRegularizer(
        traj,
        Q * ones(dim),
        name=name,
        kwargs...
    )
end

[A 0; 0 B]
[A, B]

[A 0 0; 0 B 0; 0 0 C]
[A, B, C]

# Two points

In [ ]:
# index1 = length(initial_θs) -3 
# index1 = 1
index1 = length(initial_θs) ÷ 2
index2 = length(initial_θs)

println("Co-optimizing $(initial_θs[index1] / π)π and $(initial_θs[index2] / π)π...\n")

problem1 = initial_probs[index1]
problem2 = initial_probs[index2]

dual_sys = problem1.system ⊕ problem2.system
dual_traj = problem1.trajectory ⊕ problem2.trajectory

# Construct integrators
dual_integ = [
    # UnitaryPadeIntegrator(dual_sys, :Ũ⃗, :a, order=10, autodiff=false),
    UnitaryPadeIntegrator(:Ũ⃗, :a, dual_traj, dual_sys;
        order=10,
        autodiff=false,
        calculate_pade_operators_structure=true,
        G_function=nothing,
    ),
    DerivativeIntegrator(:a, :da, dual_traj),
    DerivativeIntegrator(:da, :dda, dual_traj),
]

# Rebuild trajectory constraints
build_trajectory_constraints = true
dual_constr = AbstractConstraint[]

final_fidelity = 0.9999
fidelity_constraint = FinalUnitaryFidelityConstraint(
    :Ũ⃗,
    final_fidelity,
    dual_traj,
    hessian=false
)
push!(dual_constr, fidelity_constraint)

# Build the objective function
Q = 10.
R = 1e-2
dual_J = UnitaryPairwiseQuadraticRegularizer(dual_traj, 1)
# dual_J += UnitaryInfidelityObjective(:Ũ⃗, dual_traj, Q)
dual_J += QuadraticRegularizer(:a, dual_traj, R)
dual_J += QuadraticRegularizer(:da, dual_traj, R)
dual_J += QuadraticRegularizer(:dda, dual_traj, R)

# Ipopt options
ops = Options()
ops.recalc_y = "yes"
ops.recalc_y_feas_tol = 1e-2
ops.print_level = 5
ops.hessian_approximation = "limited-memory"

dual_p = QuantumControlProblem(
    dual_sys,
    dual_traj,
    dual_J,
    dual_integ;
    constraints=dual_constr,
    verbose=false,
    ipopt_options=ops,
    hessian_approximation=true,
    build_trajectory_constraints=build_trajectory_constraints,
)

In [ ]:
solve!(dual_p; max_iter=100)
println("Infidelity: $(1 - unitary_fidelity(dual_p))")

In [ ]:
f = Figure()
ax1 = Axis(f[1, 1])
ax2 = Axis(f[2, 1])
lines!(ax1, dual_p.trajectory[:a][1, :], color=:blue, linewidth=5)
lines!(ax1, dual_p.trajectory[:a][3, :], color=:lightblue, linewidth=5)

lines!(ax2, dual_p.trajectory[:a][2, :], color=:green, linewidth=5)
lines!(ax2, dual_p.trajectory[:a][4, :], color=:lightgreen, linewidth=5)
f

In [ ]:
interp_a = Interpolations.linear_interpolation(
    [initial_θs[index1], initial_θs[index2]],
    [dual_p.trajectory[:a][1:2, :], 
     dual_p.trajectory[:a][3:4, :]]
)

# interp_a = Interpolations.interpolate(
#     [initial_θs[index1], initial_θs[index2]],
#     [dual_p.trajectory[:a][1:2, :], 
#      dual_p.trajectory[:a][3:4, :]],
#     Gridded(Linear())
# )

interp_Θs = range(initial_θs[index1], initial_θs[index2], 50)
interp_a_vals = interp_a.(interp_Θs)

infids = map(zip(interp_Θs, interp_a_vals)) do (θ, control)
    Ũ⃗_final = unitary_rollout(
        problem1.trajectory.initial[:Ũ⃗],
        control,
        Δt,
        system
    )[:, end]
    1 - unitary_fidelity(iso_vec_to_operator(Ũ⃗_final), excitation(θ))
end

f = Figure()
lines(f[1, 1], interp_Θs, infids, color=:blue, linewidth=5)
f

# Three points

In [ ]:
# index1 = length(initial_θs) -3 
index1 = 1
index2 = length(initial_θs) ÷ 2
index3 = length(initial_θs)

println("Co-optimizing $(initial_θs[index1] / π)π, $(initial_θs[index2] / π)π, and $(initial_θs[index3] / π)π...\n")

problem1 = initial_probs[index1]
problem2 = initial_probs[index2]
problem3 = initial_probs[index3]

dual_sys = problem1.system ⊕ problem2.system ⊕ problem3.system
dual_traj = problem1.trajectory ⊕ problem2.trajectory ⊕ problem3.trajectory

# Construct integrators
dual_integ = [
    # UnitaryPadeIntegrator(dual_sys, :Ũ⃗, :a, order=10, autodiff=false),
    UnitaryPadeIntegrator(:Ũ⃗, :a, dual_traj, dual_sys;
        order=10,
        autodiff=false,
        calculate_pade_operators_structure=true,
        G_function=nothing,
    ),
    DerivativeIntegrator(:a, :da, dual_traj),
    DerivativeIntegrator(:da, :dda, dual_traj),
]

# Rebuild trajectory constraints
build_trajectory_constraints = true
dual_constr = AbstractConstraint[]

final_fidelity = 0.9999
fidelity_constraint = FinalUnitaryFidelityConstraint(
    :Ũ⃗,
    final_fidelity,
    dual_traj,
    hessian=false
)
push!(dual_constr, fidelity_constraint)

# Build the objective function
R = 1e-5
Q_pair = ones(problem1.trajectory.dims[:Ũ⃗])
Inds = ones(length(Q_pair))
Zs = zeros(length(Q_pair))
Ũ⃗1_inds = findall(Inds ⊕ Zs ⊕ Zs .≈ 1)
Ũ⃗2_inds = findall(Zs ⊕ Inds ⊕ Zs .≈ 1)
Ũ⃗3_inds = findall(Zs ⊕ Zs ⊕ Inds .≈ 1)
dual_J = UnitaryPairwiseQuadraticRegularizer(
    Q_pair, 
    1:problem1.trajectory.T,
    Ũ⃗1_inds,
    Ũ⃗2_inds
)

dual_J += UnitaryPairwiseQuadraticRegularizer(
    Q_pair, 
    1:problem1.trajectory.T,
    Ũ⃗2_inds,
    Ũ⃗3_inds
)
dual_J = UnitaryPairwiseQuadraticRegularizer(dual_traj, 1)
# dual_J += UnitaryInfidelityObjective(:Ũ⃗, dual_traj, Q)
dual_J += QuadraticRegularizer(:a, dual_traj, R)
dual_J += QuadraticRegularizer(:da, dual_traj, R)
dual_J += QuadraticRegularizer(:dda, dual_traj, R)

# Ipopt options
ops = Options()
ops.recalc_y = "yes"
ops.recalc_y_feas_tol = 1e-2
ops.print_level = 5
ops.hessian_approximation = "limited-memory"

dual_p = QuantumControlProblem(
    dual_sys,
    dual_traj,
    dual_J,
    dual_integ;
    constraints=dual_constr,
    verbose=false,
    ipopt_options=ops,
    hessian_approximation=true,
    build_trajectory_constraints=build_trajectory_constraints,
)

In [ ]:
solve!(dual_p; max_iter=100)

In [ ]:
f = Figure()
ax1 = Axis(f[1, 1])
ax2 = Axis(f[2, 1])
lines!(ax1, dual_p.trajectory[:a][1, :], color=:lightblue, linewidth=5)
lines!(ax1, dual_p.trajectory[:a][3, :], color=:blue, linewidth=5)
lines!(ax1, dual_p.trajectory[:a][5, :], color=:darkblue, linewidth=5)

lines!(ax2, dual_p.trajectory[:a][2, :], color=:lightgreen, linewidth=5)
lines!(ax2, dual_p.trajectory[:a][4, :], color=:green, linewidth=5)
lines!(ax2, dual_p.trajectory[:a][6, :], color=:darkgreen, linewidth=5)

f